## Importing

In [1]:
import os
import numpy as np
import cv2
import multiprocessing
min_confidence = 0.2

## for renaming the file

In [ ]:
%%time
path = './images/'
i = 0
for filename in os.listdir(path):
    os.rename(os.path.join(path,filename), os.path.join(path,'image_'+str(i)+'.jpg'))
    i = i +1

## class define and model load

In [2]:
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
    "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
    "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
    "sofa", "train", "tvmonitor"]
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

net = cv2.dnn.readNetFromCaffe("MobileNetSSD_deploy.prototxt.txt", "MobileNetSSD_deploy.caffemodel")

## function for detection

In [3]:
def prep(img):
    #image load
    image = cv2.imread(img)
    (h, w) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 0.007843, (300, 300), 127.5)
    return blob,h,w,image

def tini(arr, h, w, image, num):
    #load image in model
    net.setInput(arr)
    detections = net.forward()

    #look for match in image
    for i in np.arange(0, detections.shape[2]):
        
        #extract confidence from detected object
        confidence = detections[0, 0, i, 2]

        #filter out low confidence objects
        if confidence > min_confidence:
            
            idx = int(detections[0, 0, i, 1])
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            
            (startX, startY, endX, endY) = box.astype("int")

            # display the prediction
            label = "{}: {:.2f}%".format(CLASSES[idx], confidence * 100)
#             print("[INFO] {}".format(label))
            cv2.rectangle(image, (startX, startY), (endX, endY), COLORS[idx], 2)
    
            y = startY - 15 if startY - 15 > 15 else startY + 15
            cv2.putText(image, label, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)

    # show the output image
    cv2.imwrite("./output/{}_output.jpg".format(num), image)

In [12]:
prep("./images/image_0.jpg")

array([[[[0.8666515 , 0.8744945 , 0.8666515 , ..., 0.8666515 ,
          0.8744945 , 0.8666515 ],
         [0.9058665 , 0.91370946, 0.9058665 , ..., 0.9058665 ,
          0.91370946, 0.9058665 ],
         [0.91370946, 0.9215525 , 0.91370946, ..., 0.91370946,
          0.9215525 , 0.91370946],
         ...,
         [0.8274365 , 0.83527946, 0.8274365 , ..., 0.75684947,
          0.7646925 , 0.75684947],
         [0.8274365 , 0.83527946, 0.8274365 , ..., 0.75684947,
          0.7646925 , 0.75684947],
         [0.7882215 , 0.7960645 , 0.7882215 , ..., 0.70194846,
          0.7254775 , 0.7254775 ]],

        [[1.866634  , 1.874477  , 1.866634  , ..., 1.866634  ,
          1.874477  , 1.866634  ],
         [1.905849  , 1.913692  , 1.905849  , ..., 1.905849  ,
          1.913692  , 1.905849  ],
         [1.913692  , 1.921535  , 1.913692  , ..., 1.913692  ,
          1.921535  , 1.913692  ],
         ...,
         [1.8274189 , 1.835262  , 1.8274189 , ..., 1.756832  ,
          1.764675  , 1.7

## Serial

In [4]:
%%time
ls=[]
for i in range(1000):
    ls.append(prep("./images/image_{}.jpg".format(i)))
    if(i%250 == 0):
        print(i,"done!")

0 done!
250 done!
500 done!
750 done!
CPU times: user 12.1 s, sys: 1.65 s, total: 13.8 s
Wall time: 24.2 s


In [5]:
%%time
for i in range(1000):
    tini(ls[i][0], ls[i][1], ls[i][2], ls[i][3], i)
    if(i%250 == 0):
        print(i,"done!")

0 done!
250 done!
500 done!
750 done!
CPU times: user 3min 27s, sys: 17.9 s, total: 3min 45s
Wall time: 50.9 s


## Parallel 8 cores

In [12]:
def parallel(k):
  if(k==1):
    for i in range(0,250):
      tini("./images/image_{}.jpg".format(i),i)
  if(k==2):
    for i in range(250,500):
      tini("./images/image_{}.jpg".format(i),i)
  if(k==3):
    for i in range(500,750):
      tini("./images/image_{}.jpg".format(i),i)
  if(k==4):
    for i in range(750,1000):
      tini("./images/image_{}.jpg".format(i),i)
  if(k==5):
    for i in range(1000,1250):
      tini("./images/image_{}.jpg".format(i),i)
  if(k==6):
    for i in range(1250,1500):
      tini("./images/image_{}.jpg".format(i),i)
  if(k==7):
    for i in range(1500,1750):
      tini("./images/image_{}.jpg".format(i),i)
  if(k==8):
    for i in range(1750,2000):
      tini("./images/image_{}.jpg".format(i),i)

In [13]:
%%time
pool = multiprocessing.Pool(processes=8)
result = pool.map(parallel, [1,2,3,4,5,6,7,8])

CPU times: user 15.3 ms, sys: 38.2 ms, total: 53.5 ms
Wall time: 53.3 s


## Parallel 4 cores

In [3]:
def parallel_4(k):
  if(k==1):
    for i in range(0,500):
      tini("./images/image_{}.jpg".format(i),i)
  if(k==2):
    for i in range(500,1000):
      tini("./images/image_{}.jpg".format(i),i)
  if(k==3):
    for i in range(1000,1500):
      tini("./images/image_{}.jpg".format(i),i)
  if(k==4):
    for i in range(1500,2000):
      tini("./images/image_{}.jpg".format(i),i)

In [ ]:
%%time
pool = multiprocessing.Pool(processes=4)
result = pool.map(parallel_4, [1,2,3,4])